# Coroutines through generators

There are many tutorials out there showing how to use generators in Python, but most of these just show off their ability to produce data that can be looped over, such as infinite counters:

In [1]:
def counter(start=0, step=1):
    while True:
        yield start
        start += step

for ctr, char in zip(counter(), 'abcdefg'):
    print('{}: {}'.format(ctr, char))

0: a
1: b
2: c
3: d
4: e
5: f
6: g


This is a great use case for generators, but it isn't the only one.  Over time I've found that the coroutine interface is very useful for modeling complex interactions.  First, we need to address what a coroutine in Python is and how to use it.

Coroutines are a special kind of generator, one that allows for data to be passed in and acted upon without requiring all that data up-front.  Instead of arguments to your function, it's provided incrementally.  As a simple example, consider the following script:

In [4]:
def say_hello():
    print('What is your name?', end='  ')
    name = input()
    print('What year were you born?', end='  ')
    year_str = input()
    if not year_str.isdigit():
        print("That's not a real year!")
    else:
        print('Hello, {}!  You are {} years old'.format(name, 2015 - int(year_str)))

say_hello()

What is your name?  Aaron
What year were you born?  1991
Hello, Aaron!  You are 24 years old


How would you test this function?  How would you abstract out the IO so that you can run the tests automatically?  Sure, you could override `sys.stdin` and `sys.stdout`, making sure to reset them to `sys.__stdin__` and `sys.__stdout__` after you're done, but that only works because we're working with the `std*` files, in real applications user interaction is much more complicated.

Say you have a new application where the user needs to select options on multiple screens before their new account is created.  You also need to support multiple user languages.  Next, your boss requests that you add the ability to batch import new users from a CSV file since BigClient wants to enroll all their employees at once.  You could write your method to pull in the localizations, and then another method with similar logic for processing a CSV file, wrapping it up in a class to minimize code duplication:

In [6]:
class Enroller:
    def _enroll_user(self, name, email, password, initial_preferences):
        new_user = User(name, email, password, initial_preferences)
        new_user.validate()
        self.db.insert_user(new_user)
    
    def enroll_user_stdio(self):
        name = input(self.translate('name'))
        email = input(self.translate('email'))
        password = input(self.translate('password'))
        initial_preferences = input(self.translate('light_or_dark_theme'))
        self._enroll_user(name, email, password, initial_preferences)
    
    def enroll_users_from_csv(self, filename):
        with open(filename) as f:
            f.readline() # Skip the header, do we know what order the records are in?
            for line in f:
                name, email, password, initial_preferences = line.split(',')
                self._enroll_user(name, email, password, initial_preferences)

This isn't the worst code on the planet, but there are some obvious problems.  We're mixing user interaction and file IO with providing translations and inserting records.  We also are assuming the order of the records in the file, and assuming that we will always have all the fields available when reading from a file.  It also always commits to the database, meaning that has to be mocked just to test this logic.  Instead, we could use a coroutine that provides the business logic and we can have multiple "interpreters" on it:

In [ ]:
import pandas as pd


def enroll_user():
    name = yield 'name'
    email = yield 'email'
    password = yield 'password'
    initial_preferences = yield 'initial_preferences'
    new_user = User(name, email, password, initial_preferences)
    new_user.validate()
    yield new_user

class Enroller:
    def enroll_user_stdio(self):
        enroll = enroll_user()
        prompt = enroll.send()  # First call to .send (or .next) gets to the first yield
        while not isinstance(prompt, User):
            response = input(self.translate(prompt))
            prompt = enroll.send(response)
        self.db.insert_user(new_user)
    
    def enroll_users_from_csv(self, filename):
        data = pd.read_csv(filename)
        for row in data:
            enroll = enroll_user()
            prompt = enroll.send()
            while not isinstance(prompt, User):
                try:
                    response = row[prompt]
                    prompt = enroll.send(response)
                except:
                    pass